In [55]:
import numpy as np
import re
import json
import pandas as pd
from pandas import json_normalize
import matplotlib
from matplotlib import pyplot
from PIL import Image
import cv2
import matplotlib.colors as colors
from skimage import transform
#from scipy.ndimage.measurements import center_of_mass

In [56]:
def loadData(path):
    with open(path,'r') as handle:
        text_data = handle.read()
        json_data = json.loads(text_data)
        df = pd.DataFrame(json_data)
    return df

def openfile(path):
    with open(path, 'r') as handle:
        text_data = handle.read()
        text_data = '[' + re.sub(r'\}\s\{','},{', text_data) + ']'
        json_data = json.loads(text_data)
        json_data = LabelFrame(json_data)
    return json_data

#Adding frame numbers for time series plots
def LabelFrame(data):   
    df = pd.DataFrame(data)
    for i in range(len(df.ObjectList.index)):
        for j in df['ObjectList'][i]:
            j['Frame'] = i
            
    df.to_json('temp.json',orient = 'records')
    with open('temp.json') as file:
        data2 = file.read()
        jasondata = json.loads(data2)
    return jasondata

def labelData(df, x1, x2, y1, y2, label):
    df['ObjType'] = df['ObjType'].map(lambda x : 0)

    for index, row in df.iterrows():
        if ((row['Coordinates.X'] > x1) & (row['Coordinates.X'] < x2) 
        & (row['Coordinates.Y'] > y1) & (row['Coordinates.Y'] < y2)):
            df.at[index, 'ObjType'] = label
        else:
            df.at[index, 'ObjType'] = 0
    return df

#Seperating pk array values into columns
def mergedframe1(df_1):
        frames = [df_1]
        df_merged = pd.concat(frames,ignore_index = True)
        df_classify = pd.DataFrame()
        df_classify[['ObjType','pkArray','dpplIdx']] = df_merged[['ObjType','pkArray','dpplIdx']]
        
        df_classify['pkArray'] = df_classify['pkArray'].apply(lambda pkarray:np.ravel(np.transpose(pkarray)))
        
        all_list = []

        for i in range(35):
            for j in range(len(df_classify.index)):
                all_list.append(df_classify.pkArray[j][i])
        chunk = [all_list[i:i+len(df_classify.index)] for i in range(0, len(all_list), len(df_classify.index))]
        df_classify1 = pd.DataFrame(chunk)
        dfclassify=df_classify1.transpose()
        dfclassify['ObjType'] = df_classify['ObjType']
        dfclassify['rangeIdx'] = df_merged['rangeIdx']
        dfclassify['dpplIdx'] = df_merged['dpplIdx']
        dfclassify['pkArray'] = df_merged['pkArray']
        dfclassify['Range'] = df_merged['Range']
        dfclassify['Coordinates.X'] = df_merged['Coordinates.X']
        dfclassify['Coordinates.Y'] = df_merged['Coordinates.Y']
        dfclassify["pkValue1.Real"] = df_merged["pkValue1.Real"]
        dfclassify["pkValue1.Imaginary"] = df_merged['pkValue1.Imaginary']
        dfclassify["pkValue2.Real"] = df_merged["pkValue2.Real"]
        dfclassify["pkValue2.Imaginary"] = df_merged['pkValue2.Imaginary']
        dfclassify["pkValue3.Real"] = df_merged["pkValue3.Real"]
        dfclassify["pkValue3.Imaginary"] = df_merged['pkValue3.Imaginary']
        dfclassify["pkValue4.Real"] = df_merged["pkValue4.Real"]
        dfclassify["pkValue4.Imaginary"] = df_merged['pkValue4.Imaginary']
        
        dfclassify['Frame'] = df_merged['Frame']
        dfclassify['Range'] = df_merged['Range']
        return dfclassify

# Load all files from json to a dataframe after 
def startdf(json_data,Label):
    df_1= pd.DataFrame()
    df1 = pd.DataFrame((json_data))
    for j in range(len(df1.index)):
        df2 = json_normalize(df1['ObjectList'][j])
        df2['ObjType'] = df2['ObjType'].map(lambda ObjType: Label)        
        #df2 = df2[df2.Range <= UIdx]
        #df2 = df2[df2.Range >= LIdx]
        #df2 = df2[df2.dpplIdx == 1]   
        df_1 = pd.concat([df2,df_1],ignore_index =True)
    df_1 = df_1.sort_values(by=['Frame'],ascending = True)
    df_1 = df_1.reset_index(drop=True)
    df_1['pkArray'] = df_1['pkArray'].map(lambda x:np.transpose(x))
    df_final = mergedframe1(df_1)
    return df_final

def padHumanRows(df):
  for ind, row in df.iterrows():
    if ((ind % 10) == 0):
      count = 0
      while ((count < 10) and (df.iloc[ind + count]['Frame'] == int(ind/10))):
        count = count + 1
        if ((ind + count) == len(df.index)):
          break
      while (count < 10):
        row = pd.DataFrame({"0": 0, "1": 0, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "8": 0, "9": 0, "10": 0, "11": 0, "12": 0, "13": 0, "14": 0, "15": 0, "16": 0, "17": 0, "18": 0, "19": 0, "20": 0, "21": 0, "22": 0, "23": 0, "24": 0, "25": 0, "26": 0, "27": 0, "28": 0, "29": 0, "30": 0, "31": 0, "32": 0, "33": 0, "34": 0, "rangeIdx":0, "dpplIdx":0, "Frame": ind/10, "ObjType":0}, index=[ind+count-1+0.5])
        df = df.append(row, ignore_index=False)
        df = df.sort_index().reset_index(drop=True)
        df = df.reindex(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31", "32", "33", "34", "rangeIdx", "dpplIdx", "Frame", "ObjType"], axis=1)
        count = count + 1
  return df

def padCarRows(df):
  for ind, row in df.iterrows():
    if ((ind % 10) == 0):
      count = 0
      while ((count < 10) and (df.iloc[ind + count]['Frame'] == int(ind/10))):
        count = count + 1
        if ((ind + count) == len(df.index)):
          break
      while (count < 10):
        row = pd.DataFrame({0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, "rangeIdx":0, "dpplIdx":0, "Frame": ind/10, "ObjType":0}, index=[ind+count-1+0.5])
        df = df.append(row, ignore_index=False)
        df = df.sort_index().reset_index(drop=True)
        df = df.reindex([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, "rangeIdx", "dpplIdx", "Frame", "ObjType"], axis=1)
        count = count + 1
  return df


def padRows(df, obj):
  if (obj == "car"):
    while (df.shape[0] != ((df['Frame'].max() + 1)*10)):
      df = padCarRows(df)
  else:
    while (df.shape[0] != ((df['Frame'].max() + 1)*10)):
      df = padHumanRows(df)  
  return df


def dfTo2DArray(df):
  df = df[df.columns[0:38]].values
  return df

#to correct the dppl idx
def correctDpplIdx(x):
    if x <=31:
        return x
    if x>31:
        return x-64


# frameTo128x64() function converts a 2D input array
# where rows == objects in json file and cols == 0,1,..34 peak array elements, rangeIdx, dopplIdx, Frame.
# The 2D array is converted to 3D array of size no. of frames * 128 * 64

def frameTo128x64(arr):
    count = 0
    new3DArray = np.empty((1, 128, 64))
    arr128x64 = np.zeros((128, 64))
    for row in range(0, np.shape(arr)[0], 1):
        if count<=10:
            rangeIndex = int(arr[row][35])
            dopplerIndex = int(correctDpplIdx(arr[row][36]))
            r = 0
            
            for rangeID in range(rangeIndex-3, rangeIndex+4, 1):
                d = 0
                for dopplID in range(dopplerIndex-2, dopplerIndex+3, 1):
                    if ((0 <= rangeID <= 127) and (-32 <= dopplID <= 31)):
                        if(arr128x64[rangeID][dopplID+32] < arr[row][(d * 7) + r]):
                            arr128x64[rangeID][dopplID+32] = arr[row][(d * 7) + r]
                    d = d + 1
                r = r + 1
        count= count + 1
        if count == 10:
            count = 0
            arr128x64 = np.reshape(arr128x64, (1, 128, 64))
            new3DArray = np.append(new3DArray, arr128x64, axis=0)
            arr128x64 = np.zeros((128, 64))
    return new3DArray[1:, :, :]


# plot arrays/images in dataset
def plotArrays(radarInp, title):
    n_samples = 5

    inp = radarInp

    pyplot.figure(figsize=(50, 20))

    for i in range(n_samples):
        pyplot.subplot(2, n_samples, 1 + i)
        pyplot.axis([0, 64, 0, 128])
        pyplot.xlabel("doppler index")
        pyplot.ylabel("range index(res=0.5m)")
        pyplot.title(title)
        pyplot.imshow(radarInp[i+100].astype('uint8'))
        
    pyplot.show()

In [87]:
def funcTransform(matrix):
        scaling_factor = 1
        min_value, max_value = np.min(matrix), np.max(matrix)
        cmap = pyplot.get_cmap('plasma')
        norm = colors.Normalize(vmin=min_value, vmax=max_value)
        colored_matrix = cmap(norm(matrix))
        colored_matrix = colored_matrix[:, :, :3]
        x_shape, y_shape, _ = colored_matrix.shape
        resized_matrix = transform.resize(colored_matrix,
                                          (scaling_factor*x_shape,
                                           scaling_factor*y_shape, 3),
                                          order=0)
        color_scale_params = [cmap, norm]
        return resized_matrix, color_scale_params

    
def getBoundingBoxCoord(img, y, seqName, i):
    #img = cv2.imread("radarImage.png")

    # set up initial coordinates for the tracking window
    x = 32

    # Set up initial size of the tracking window
    height, width = 1, 1
    track_window = (x,y,width,height)

    # set up region of interest (roi)
    roi = img[y:y+height, x:x+width]

    # apply mask
    hsv_img =  cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_img, np.array((0, 20, 20)), np.array((180, 250, 250)))
    hist_img = cv2.calcHist([hsv_img], [0], mask, [180], [0,180])
    cv2.normalize(hist_img, hist_img, 0, 255, cv2.NORM_MINMAX)

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv],[0],hist_img,[0,180],1)

    # Setup the termination criteria: 10 iteration 1 pxl movement
    term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 50, 1 )

    # apply camshift to get the new location
    ret, track_window = cv2.CamShift(dst, track_window, term_crit)

    # Draw a box around the ROI
    pts1 = cv2.boxPoints(ret)
    bX = np.int0(min(pts1[0][0], pts1[1][0], pts1[2][0], pts1[3][0]))
    bY = np.int0(min(pts1[0][1], pts1[1][1], pts1[2][1], pts1[3][1]))
    bW = np.int0(max(pts1[0][0], pts1[1][0], pts1[2][0], pts1[3][0]) - bX)
    bH = np.int0(max(pts1[0][1], pts1[1][1], pts1[2][1], pts1[3][1]) - bY)
    pts2 = np.int0([[bX, bY], [bX, bY+bH], [bX+bW, bY+bH], [bX+bW, bY]])
    img2 = cv2.polylines(img, [pts2], True, 255,2)
    pyplot.imsave("radarImages/annotatedImages/{seq}_image{num}.png".format(seq = seqName, num=i), img2)

    #pyplot.subplot(1, 1, 1)
    #pyplot.imshow(img2)
    #pyplot.show()
    
    return [int(bX), int(bY), int(bW), int(bH)]

In [58]:
# humanDay1_2pm = loadData("../RadarDataset/human/human_2pm.json")
# humanDay1_5pm = loadData("../RadarDataset/human/human_5pm.json")
# humanDay1_9pm = loadData("../RadarDataset/human/human_9pm.json")
# humanDay1_3pm = loadData("../RadarDataset/human/human_3pm.json")
# humanDay1_6pm = loadData("../RadarDataset/human/human_6pm.json")

In [59]:
carVday2_5 = startdf(openfile("../RadarDataset/carV/Car Vertical_5_28_02_2020 12_38_17.json"), 2)
carVday1_6 = startdf(openfile("../RadarDataset/carV/Car Vertical_6_27_01_2020 12_42_51.json"), 2)
carVday1_10 = startdf(openfile("../RadarDataset/carV/Car Vertical_10_27_01_2020 12_46_59.json"), 2)
carVday2_11 = startdf(openfile("../RadarDataset/carV/Car Vertical_11_28_02_2020 13_00_06.json"), 2)
carVday2_15 = startdf(openfile("../RadarDataset/carV/Car Vertical_15_28_02_2020 13_23_56.json"), 2)
carVday1_16 = startdf(openfile("../RadarDataset/carV/Car Vertical_16_27_01_2020 12_53_52.json"), 2)
carVday2_20 = startdf(openfile("../RadarDataset/carV/Car Vertical_20_28_02_2020 15_38_52.json"), 2)
carVday1_19 = startdf(openfile("../RadarDataset/carV/Car Vertical_19_27_01_2020 12_57_35.json"), 2)

In [60]:
carHday1_5 = startdf(openfile("../RadarDataset/carH/Car Horizontal_5_27_01_2020 13_05_27.json"), 3)
carHday1_7 = startdf(openfile("../RadarDataset/carH/Car Horizontal_7_27_01_2020 13_07_36.json"), 3)
carHday1_11 = startdf(openfile("../RadarDataset/carH/Car Horizontal_11_27_01_2020 13_40_38.json"), 3)
carHday1_17 = startdf(openfile("../RadarDataset/carH/Car Horizontal_17_27_01_2020 13_17_50.json"), 3)
carHday1_20 = startdf(openfile("../RadarDataset/carH/Car Horizontal_20_27_01_2020 13_21_11.json"), 3)
carHday2_2 = startdf(openfile("../RadarDataset/carH/Car Horizontal_2_12_03_2020 14_50_24.json"), 3)
carHday2_4 = startdf(openfile("../RadarDataset/carH/Car Horizontal_4_12_03_2020 14_53_38.json"), 3)
carHday2_6 = startdf(openfile("../RadarDataset/carH/Car Horizontal_6_12_03_2020 15_00_25.json"), 3)

In [61]:
# arrHumanDay1_2pm = dfTo2DArray(padRows(labelData(humanDay1_2pm, -0.5, 0.5, 0.1, 7.9, 1), "human"))
# arrHumanDay1_5pm = dfTo2DArray(padRows(labelData(humanDay1_5pm, -0.5, 0.5, 0.1, 7.9, 1), "human"))
# arrHumanDay1_9pm = dfTo2DArray(padRows(labelData(humanDay1_9pm, -0.5, 0.5, 0.1, 7.9, 1), "human"))
# arrHumanDay1_3pm = dfTo2DArray(padRows(labelData(humanDay1_3pm, -0.5, 0.5, 0.1, 7.9, 1), "human"))
# arrHumanDay1_6pm = dfTo2DArray(padRows(labelData(humanDay1_6pm, -0.5, 0.5, 0.1, 7.9, 1), "human"))

In [71]:
arrCarVday1_6 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_6, -1.5, 1.5, 5.5, 7.5, 2), "car")))
arrCarVday1_10 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_10, -1.5, 1.5, 9.5, 11.5, 2), "car")))
arrCarVday1_16 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_16, -1.5, 1.5, 15.5, 17.5, 2), "car")))
arrCarVday1_19 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday1_19, -1.5, 1.5, 18.5, 20.5, 2), "car")))
arrCarVday2_5 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_5, -1.5, 1.5, 4.5, 6.5, 2), "car")))
arrCarVday2_15 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_15, -1.5, 1.5, 14.5, 16.5, 2), "car")))
arrCarVday2_11 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_11, -1.5, 1.5, 10.5, 12.5, 2), "car")))
arrCarVday2_20 = frameTo128x64(dfTo2DArray(padRows(labelData(carVday2_20, -1.5, 1.5, 19.5, 21.5, 2), "car")))

In [72]:
arrCarHday1_5 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_5, -3, 3, 4.5, 7, 3), "car")))
arrCarHday1_7 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_7, -3, 3, 6.5, 9, 3), "car")))
arrCarHday1_11 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_11, -3, 3, 10.5, 13, 3), "car")))
arrCarHday1_17 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_17, -3, 3, 16.5, 19, 3), "car")))
arrCarHday1_20 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday1_20, -3, 3, 19.5, 22, 3), "car")))
arrCarHday2_2 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday2_2, -3, 3, 1.5, 4, 3), "car")))
arrCarHday2_4 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday2_4, -3, 3, 3.5, 6, 3), "car")))
arrCarHday2_6 = frameTo128x64(dfTo2DArray(padRows(labelData(carHday2_6, -3, 3, 5.5, 8, 3), "car")))

In [79]:
#plotArrays(arrCarHday1_11, "Radar Maps of human moving vertically 1 to 7m")
np.shape(arrCarVday1_6)

(345, 128, 64)

In [88]:
def createJsonAnnotation(imgID, annoID, bbPts, catID, imgName):
    image = {
        "id": imgID,
        "width": 64,
        "height": 128,
        "file_name": imgName,
    }
    annotation = {
        "image_id": imgID,
        "category_id": catID,
        "id": annoID,
        "bbox": bbPts,
    }
    return image, annotation

def boxAnnotation(carSeq, y, seqName, catID, imgID, annotationID):
    for i in range(0, len(carSeq)):
        image, c_param = funcTransform(carSeq[i])
        pyplot.imsave("radarImages/rawImages/{seq}_image{num}.png".format(seq = seqName, num=i), image)
        radarImageName = "{seq}_image{num}.png".format(seq = seqName, num=i)
        img = cv2.imread("radarImages/rawImages/{seq}_image{num}.png".format(seq = seqName, num=i))
        bbPts = getBoundingBoxCoord(img, y, seqName, i)
        imgJSON, annotateJSON = createJsonAnnotation(imgID, annotationID, bbPts, catID, radarImageName)
        images.append(imgJSON)
        imgID += 1
        annotations.append(annotateJSON)
        annotationID += 1
    return imgID, annotationID

In [89]:
info = [{
    "year": 2021,
    "version": 1.0,
    "description": "UAY-IITH dataset",
    "url": "",
    }]

categories = [{
        "id": 1,
        "name": "human",
    },
    {
        "id": 2,
        "name": "carVertical",
    },
    {
        "id": 3,
        "name": "carHorizontal",
    }]

images = []
annotations = []
imgID = 1
annotationID = 1

imgID, annotationID = boxAnnotation(arrCarVday1_6, 12, "arrCarVday1_6", 2, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarVday1_10, 20, "arrCarVday1_10", 2, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarVday1_16, 32, "arrCarVday1_16", 2, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarVday1_19, 38, "arrCarVday1_19", 2, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarVday2_5, 10, "arrCarVday2_5", 2, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarVday2_11, 22, "arrCarVday2_11", 2, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarVday2_20, 40, "arrCarVday2_20", 2, imgID, annotationID)

imgID, annotationID = boxAnnotation(arrCarHday1_5, 10, "arrCarHday1_5", 3, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarHday1_7, 14, "arrCarHday1_7", 3, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarHday1_11, 22, "arrCarHday1_11", 3, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarHday1_17, 34, "arrCarHday1_17", 3, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarHday1_20, 40, "arrCarHday1_20", 3, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarHday2_2, 4, "arrCarHday2_2", 3, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarHday2_4, 8, "arrCarHday2_4", 3, imgID, annotationID)
imgID, annotationID = boxAnnotation(arrCarHday2_6, 12, "arrCarHday2_6", 3, imgID, annotationID)

annotationFile = {"info": info, "categories": categories, "images": images, "annotations": annotations}

with open("radarImages/UAY_Dataset.json", "w") as outfile:
    json.dump(annotationFile, outfile, indent=4)